In [1]:
from flask import Flask, render_template, jsonify, request
from pymongo import MongoClient
from bson.json_util import dumps
import json
from datetime import datetime, timedelta
import os

app = Flask(__name__)

# MongoDB connection - replace with your connection string
# You can use environment variables for sensitive data
MONGO_URI = os.environ.get('MONGO_URI', 'mongodb://localhost:27017/')
DB_NAME = os.environ.get('DB_NAME', 'portfolio_management')

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client[DB_NAME]

# Collections
stocks_collection = db['stock_prices']
portfolio_collection = db['portfolio']
sentiment_collection = db['company_news']
predictions_collection = db['predictions']

In [3]:

db['stock_predictions'].find_one()

{'_id': ObjectId('6811b24854fd7a8471f2b13f'),
 'company': 'AAPL',
 'predicted_price': 221.6172332763672,
 'date': datetime.datetime(2025, 4, 30, 5, 16, 56, 506000),
 'model': 'LSTM',
 'created_at': datetime.datetime(2025, 4, 30, 5, 16, 56, 508000)}

In [24]:
import statistics

company_name = "Apple"
ticker = "AAPL"

# Query all documents for the specific company
cursor = sentiment_collection.find({"company": company_name})

# Initialize containers
dates = []
sentiment_scores = []
positive_count = 0
negative_count = 0
neutral_count = 0

# Process documents
for doc in cursor:
    sentiment = doc.get("sentiment", {})
    score = sentiment.get("score")
    label = sentiment.get("label")

    # Use week_start as the reference date
    date = doc.get("week_start")
    if score is not None and date is not None:
        dates.append(date.strftime("%Y-%m-%d"))
        sentiment_scores.append(score if label != "NEGATIVE" else -score)

        if label == "POSITIVE":
            positive_count += 1
        elif label == "NEGATIVE":
            negative_count += 1
        elif label == "NEUTRAL":
            neutral_count += 1

# Compute average sentiment
avg_sentiment = round(statistics.mean(sentiment_scores), 3) if sentiment_scores else 0.0

# Final structure
result = {
    "ticker": ticker,
    "dates": dates,
    "sentiment_scores": sentiment_scores,
    "avg_sentiment": avg_sentiment,
    "positive_count": positive_count,
    "negative_count": negative_count,
    "neutral_count": neutral_count
}

print(result)

{'ticker': 'AAPL', 'dates': ['2025-04-22', '2025-04-22', '2025-04-22', '2025-04-22', '2025-04-22', '2025-04-22', '2025-04-15', '2025-04-15', '2025-04-15', '2025-04-15', '2025-04-15', '2025-04-15', '2025-04-08', '2025-04-08', '2025-04-08', '2025-04-08', '2025-04-08', '2025-04-01', '2025-04-01', '2025-04-01', '2025-04-01', '2025-04-01', '2025-04-01', '2025-04-01', '2025-04-22', '2025-04-22', '2025-04-22', '2025-04-22', '2025-04-22', '2025-04-22'], 'sentiment_scores': [-0.9975680708885193, -0.714809000492096, 0.9531815052032471, -0.9665292501449585, -0.9913508892059326, -0.999424934387207, -0.9981364011764526, -0.997789740562439, 0.9970389604568481, -0.9975489974021912, 0.9996134638786316, 0.5490578413009644, -0.9698610901832581, 0.9738195538520813, -0.7662594318389893, -0.9756479263305664, -0.9994620680809021, -0.9966101050376892, -0.9972981810569763, -0.9972981810569763, 0.9738195538520813, -0.9808306694030762, 0.9982143640518188, -0.9989649057388306, -0.9975680708885193, -0.71480900049

In [25]:
cursor = sentiment_collection.find({"company": 'Apple'})

In [26]:
cursor[0]

{'_id': ObjectId('68110218353171b797e312e2'),
 'company': 'Apple',
 'headline': 'The EU isn’t happy with Apple’s tax on alternative app stores',
 'source': 'The Verge',
 'url': 'https://www.theverge.com/news/636196/apple-eu-dma-probe-alternative-app-stores-tax',
 'published_at': '2025-04-23T11:30:48Z',
 'week_start': datetime.datetime(2025, 4, 22, 0, 0),
 'week_end': datetime.datetime(2025, 4, 29, 0, 0),
 'created_at': datetime.datetime(2025, 4, 29, 16, 45, 12, 582000),
 'content': 'The European Commission has also closed its investigation into the iPhones browser choice screen.\r\nThe European Commission has also closed its investigation into the iPhones browser choice screen.\r\nT… [+2572 chars]',
 'sentiment': {'label': 'NEGATIVE', 'score': 0.9975680708885193}}